#### seasonal extent change

In [1]:
import ee
import geemap
import geopandas as gpd

c:\Users\jtrum\miniconda3\envs\wash\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\jtrum\AppData\Local\Temp\ipykernel_38656\3826985800.py:3: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. Howeve

In [2]:
ee.Authenticate()
ee.Initialize()

In [3]:
aoi = gpd.read_file('C:/Users/jtrum/Wash scan/data/aoiLuanda.geojson')

In [4]:
# Define the extent (AOI) for Luanda
luanda_geometry = ee.Geometry.Point([13.2344, -8.8115]) 
luanda_extent = ee.Geometry.Rectangle([12.8, -9.1, 13.6, -8.5])  # Replace with the actual bounding box coordinates
# Set the location, AOI, and ID
pt = luanda_geometry
AOI = luanda_extent
id = "L1a"

# Create Water Mask
water_mask_id = 'JRC/GSW1_0/GlobalSurfaceWater'
WaterMask = ee.Image(water_mask_id)
water_mask = WaterMask.select('transition')
blank = ee.Image(0)
non_water = blank.addBands(water_mask).unmask().select('transition').eq(0).rename('non_water')

# Create Slope Mask
srtm_id = 'NASA/NASADEM_HGT/001'
srtm = ee.Image(srtm_id)
slope = ee.Terrain.slope(srtm)
grade_15 = slope.gt(15)
gt_grade_15 = grade_15.updateMask(grade_15.neq(0))
slope_mask = gt_grade_15.clip(AOI).unmask(0).subtract(1).multiply(-1)

# Load Sentinel-1 C-band SAR Ground Range collection (log scaling, VV co-polar)
collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
              .filterBounds(pt)
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
              .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
              .select('VV'))

# Filter by date

# Flood date: 03/13/2017 03/28/2017
during_flood_collection = (collection
                            .filterDate('2017-03-14', '2017-03-29'))
during_flood = (during_flood_collection
                .first()
                .clip(AOI)
                .updateMask(non_water)
                .updateMask(slope_mask))
print('During Flood Image Collection: ', during_flood_collection)

# Before Flood & in Dry Season: May - Sept
before_flood_collection = (collection
                            .filterDate('2016-08-01', '2016-09-30'))
before_flood = (before_flood_collection
                .median()
                .clip(AOI)
                .updateMask(non_water)
                .updateMask(slope_mask))
print('Before Flood Image Collection: ', before_flood_collection)

def ymd_list(image_collection):
    # Define a function to iterate over the image collection
    def iter_func(image, new_list):
        date = ee.Number.parse(ee.Date(image.date()).format("YYYYMMdd"))
        new_list = ee.List(new_list)
        return ee.List(new_list.add(date).sort())

    # Use list comprehension to iterate over the image collection
    return ee.List(image_collection.iterate(iter_func, ee.List([])))

# Print the image dates for Before Flood and During Flood collections
print('Before Flood Image Dates: ', ymd_list(before_flood_collection))
print('During Flood Image Dates: ', ymd_list(during_flood_collection))

# Set smoothing box (30m is 3x size of S1 pixel)
smoothing_box = 30



During Flood Image Collection:  ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.filter",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.map",
          "arguments": {
            "baseAlgorithm": {
              "functionDefinitionValue": {
                "argumentNames": [
                  "_MAPPING_VAR_0_0"
                ],
                "body": {
                  "functionInvocationValue": {
                    "functionName": "Image.select",
                    "arguments": {
                      "bandSelectors": {
                        "constantValue": [
                          "VV"
                        ]
                      },
                      "input": {
                        "argumentReference": "_MAPPING_VAR_0_0"
                      }
                    }
                  }
                }
              }
            },
            "collection":

In [5]:

# Create difference images (during flood - before flood)
difference_smoothed = (during_flood
                       .focal_median(radius=smoothing_box, kernelType='square', units='meters')
                       .subtract(before_flood.focal_median(radius=smoothing_box, kernelType='square', units='meters')))
difference_raw = during_flood.subtract(before_flood)


In [6]:
# Check if difference_smoothed and AOI are defined and contain data
if difference_smoothed is not None and AOI is not None:
    # Check if the image has bands and AOI has coordinates
    if difference_smoothed.bandNames().size().getInfo() > 0 and AOI.coordinates().size().getInfo() > 0:
        # Clip the image
        difference_smoothed_clipped = difference_smoothed.clip(AOI)
        # Continue with further operations
    else:
        print("Either difference_smoothed or AOI is empty. Cannot clip image.")
else:
    print("Either difference_smoothed or AOI is not properly defined. Cannot clip image.")


EEException: Image.clip: Parameter 'input' is required.

In [ ]:
# Check if difference_smoothed is defined and print its information
if difference_smoothed is not None:
    print("difference_smoothed information:", difference_smoothed.getInfo())
else:
    print("difference_smoothed is not properly defined.")

# Check if AOI (luanda_extent) is defined and print its information
if AOI is not None:
    print("AOI (luanda_extent) information:", AOI.getInfo())
else:
    print("AOI (luanda_extent) is not properly defined.")

# Clip the image only if both difference_smoothed and AOI are properly defined
if difference_smoothed is not None and AOI is not None:
    difference_smoothed_clipped = difference_smoothed.clip(AOI)
    # Continue with further operations
else:
    print("Either difference_smoothed or AOI is not properly defined. Cannot clip image.")


EEException: Image.clip: Parameter 'input' is required.

In [ ]:

asset_id = 'NASA/NASADEM_HGT/001' # NASADEM elevation data

# Check asset existence
asset_info = ee.data.getInfo(asset_id)
if asset_info is not None:
    print(f"Asset '{asset_id}' exists.")
else:
    print(f"Asset '{asset_id}' does not exist or is not accessible.")

# Resample difference raster
band_VV = difference_smoothed.select('VV')
proj = band_VV.projection().getInfo()
crs = proj['crs']

# Ensure difference_smoothed is defined before attempting to resample
difference_smoothed_resampled = (difference_smoothed
                                  .resample('bilinear')
                                  .reproject({
                                      'crs': crs,
                                      'scale': 10.0
                                  })
                                  .clip(AOI))


During Flood Image Collection:  ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.filter",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.map",
          "arguments": {
            "baseAlgorithm": {
              "functionDefinitionValue": {
                "argumentNames": [
                  "_MAPPING_VAR_0_0"
                ],
                "body": {
                  "functionInvocationValue": {
                    "functionName": "Image.select",
                    "arguments": {
                      "bandSelectors": {
                        "constantValue": [
                          "VV"
                        ]
                      },
                      "input": {
                        "argumentReference": "_MAPPING_VAR_0_0"
                      }
                    }
                  }
                }
              }
            },
            "collection":

Before Flood Image Dates:  ee.List({
  "functionInvocationValue": {
    "functionName": "Collection.iterate",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.filter",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.map",
                "arguments": {
                  "baseAlgorithm": {
                    "functionDefinitionValue": {
                      "argumentNames": [
                        "_MAPPING_VAR_0_0"
                      ],
                      "body": {
                        "functionInvocationValue": {
                          "functionName": "Image.select",
                          "arguments": {
                            "bandSelectors": {
                              "constantValue": [
                                "VV"
                              ]
                            },
             

Asset 'NASA/NASADEM_HGT/001' exists.


EEException: Image.clip: Parameter 'input' is required.

In [ ]:
# Resample difference raster
band_VV = difference_smoothed.select('VV')
projection_info = band_VV.projection()
crs = projection_info.crs()

# Simplified resampling and reprojection
resampled_image = difference_smoothed.resample('bilinear').reproject(crs=crs)

# Update mask separately
resampled_image_with_mask = resampled_image.updateMask(AOI)

# Display the resampled image to check if it loads successfully
print(resampled_image_with_mask.getInfo())


EEException: A mapped function's arguments cannot be used in client-side operations